In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install pyarrow


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import zipfile 
import io
import os
import concurrent.futures
import time
import subprocess
import multiprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow
import json
from matplotlib.ticker import FormatStrFormatter

In [5]:
import sys
import os

sys.path.append(os.path.abspath('../advmla_at3_package'))

In [6]:
df_train = pd.read_feather('../data/processed/train_data.feather')
df_test = pd.read_feather('../data/processed/test_data.feather')

In [7]:
df_train.shape

(5106602, 13)

In [8]:
df_train.head(10)

,searchDate,flightDate,startingAirport,destinationAirport,isNonStop,totalFare,totalTravelDistance,segmentsArrivalAirportCode,DepartureTimeHour,CabinCode,AirlineNameScore,date_diff_days,weekday
3767896,2022-04-18,2022-05-06,LGA,DFW,True,93.599998,1380.0,DFW,8,1.0,2,18,4
929418,2022-05-11,2022-05-18,BOS,IAD,False,207.600006,406.0,JFK||IAD,11,1.0,4,7,2
5456598,2022-05-08,2022-06-21,ORD,EWR,False,358.600006,725.0,DTW||EWR,14,1.0,4,44,1
4876708,2022-05-08,2022-05-18,OAK,BOS,False,728.599976,2688.0,SLC||BOS,17,1.0,4,10,2
2004101,2022-05-12,2022-07-01,DFW,ATL,True,228.600006,725.0,ATL,19,1.0,4,50,4
1057863,2022-05-09,2022-06-29,BOS,PHL,True,93.599998,280.0,PHL,11,1.0,4,51,2
96461,2022-05-07,2022-05-17,ATL,EWR,True,213.600006,762.0,EWR,17,1.0,4,10,1
5565752,2022-05-02,2022-06-09,PHL,ORD,False,339.579987,NaN,MCO||ORD,7,1.0,1,38,3
4455243,2022-05-09,2022-06-15,MIA,LGA,True,140.600006,1104.0,LGA,12,1.0,4,37,2
3553263,2022-05-05,2022-05-15,LAX,PHL,False,821.590027,3335.0,FLL||PHL,10,1.0,2,10,6


In [9]:
df_train.isnull().sum()

searchDate                         0
flightDate                         0
startingAirport                    0
destinationAirport                 0
isNonStop                          0
totalFare                          0
totalTravelDistance           484110
segmentsArrivalAirportCode         0
DepartureTimeHour                  0
CabinCode                          0
AirlineNameScore                   0
date_diff_days                     0
weekday                            0
dtype: int64

In [10]:
df_test.isnull().sum()

searchDate                         0
flightDate                         0
startingAirport                    0
destinationAirport                 0
isNonStop                          0
totalFare                          0
totalTravelDistance           120986
segmentsArrivalAirportCode         0
DepartureTimeHour                  0
CabinCode                          0
AirlineNameScore                   0
date_diff_days                     0
weekday                            0
dtype: int64

In [7]:
from engineering.clean_master import preprocessing_data

df_train_processed, df_test_precessed = preprocessing_data(df_train, df_test)

In [8]:
selected_features = ['date_diff_days', 'weekday', 'CabinCode', 'DepartureTimeHour', 'totalTravelDistance']
categorical_features = ['startingAirport', 'destinationAirport', 'isNonStop']
target = 'totalFare'

df_filtered_train = df_train_processed[categorical_features + selected_features + [target]]
df_filtered_test = df_test_precessed[categorical_features + selected_features + [target]]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression

numerical_features = ['totalTravelDistance']

preprocessor = ColumnTransformer(
    transformers=[
        ('one-hot', OneHotEncoder(handle_unknown='ignore'), categorical_features),  
        ('scaler', StandardScaler(), numerical_features) 
    ],
    remainder='passthrough')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression()) 
])

In [10]:
from sklearn.model_selection import train_test_split


# Split the datasets
train_df, val_df = train_test_split(df_filtered_train, test_size=0.2, random_state=42)

X_train = train_df.drop(columns=[target], axis=1)  
y_train = train_df[target]

X_val = val_df.drop(columns=[target], axis=1)  
y_val = val_df[target]

X_test = df_filtered_test.drop(columns=[target], axis=1)  
y_test = df_filtered_test[target]

In [11]:
from modeling.null import NullRegressor

base_model = NullRegressor()

y_base = base_model.fit_predict(y_train)


In [12]:
# Baseline result

from modeling.performance import print_regressor_scores

print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')
print_regressor_scores(y_preds=base_model.predict(y_val), y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_preds=base_model.predict(y_test), y_actuals=y_test, set_name='Testing')

RMSE Training: 190.866455078125
MAE Training: 136.70587158203125
RMSE Validation: 189.75318908691406
MAE Validation: 136.62557983398438
RMSE Testing: 191.07180786132812
MAE Testing: 136.75062561035156


In [13]:
pipeline.fit(X_train, y_train)

y_preds = pipeline.predict(X_train)
y_val_preds = pipeline.predict(X_val)
y_test_preds = pipeline.predict(X_test)

In [14]:
print_regressor_scores(y_preds, y_train, set_name='Training')
print_regressor_scores(y_val_preds, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds, y_actuals=y_test, set_name='Testing')

RMSE Training: 136.07393438739464
MAE Training: 96.59250108703283
RMSE Validation: 135.21356822677274
MAE Validation: 96.54232007646003
RMSE Testing: 135.8049850296426
MAE Testing: 96.63959049758105


In [30]:
import joblib
model_path = 'linear_model.joblib'
joblib.dump(pipeline, model_path)

['linear_model.joblib']

In [19]:
!pip install lightgbm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
from lightgbm import LGBMRegressor

pipeline2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbm', LGBMRegressor(learning_rate=0.001, num_leaves=50, max_depth=7)) 
])

pipeline2.fit(X_train, y_train)

y_preds2 = pipeline2.predict(X_train)
y_val_preds2 = pipeline2.predict(X_val)
y_test_preds2 = pipeline2.predict(X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033747 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 413
[LightGBM] [Info] Number of data points in the train set: 4085281, number of used features: 36
[LightGBM] [Info] Start training from score 324.383684
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [21]:
print_regressor_scores(y_preds2, y_train, set_name='Training')
print_regressor_scores(y_val_preds2, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds2, y_actuals=y_test, set_name='Testing')

RMSE Training: 143.7091744298257
MAE Training: 104.79239795799226
RMSE Validation: 142.82371710025612
MAE Validation: 104.76881100929674
RMSE Testing: 143.49196418856522
MAE Testing: 104.8158550812793


In [39]:
from sklearn.linear_model import SGDRegressor

pipeline3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('SGD', SGDRegressor(learning_rate='optimal',alpha=0.1)) 
])

pipeline3.fit(X_train, y_train)

y_preds3 = pipeline3.predict(X_train)
y_val_preds3 = pipeline3.predict(X_val)
y_test_preds3 = pipeline3.predict(X_test)

In [41]:
# 3. SGDRegressor result

print_regressor_scores(y_preds3, y_train, set_name='Training')
print_regressor_scores(y_val_preds3, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds3, y_actuals=y_test, set_name='Testing')

RMSE Training: 663988798.303233
MAE Training: 527912128.62180895
RMSE Validation: 663994738.175438
MAE Validation: 528064419.2527296
RMSE Testing: 664383560.1067221
MAE Testing: 528123134.92055064


In [42]:
pipeline_sgd = Pipeline([
    ('preprocessor', preprocessor),
    ('SGD', SGDRegressor(
        learning_rate='invscaling',  
        eta0=0.001,  
        max_iter=1000,  
        tol=1e-4  
    ))
])

pipeline_sgd.fit(X_train, y_train)

y_preds3 = pipeline_sgd.predict(X_train)
y_val_preds3 = pipeline_sgd.predict(X_val)
y_test_preds3 = pipeline_sgd.predict(X_test)

In [43]:
# 3. SGDRegressor result

print_regressor_scores(y_preds3, y_train, set_name='Training')
print_regressor_scores(y_val_preds3, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds3, y_actuals=y_test, set_name='Testing')

RMSE Training: 136.7813937479851
MAE Training: 96.24451014140384
RMSE Validation: 135.9198738981014
MAE Validation: 96.18253896091979
RMSE Testing: 136.51112215924726
MAE Testing: 96.27562736429101


In [25]:
from sklearn.ensemble import AdaBoostRegressor

pipeline4 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('Ada', AdaBoostRegressor()) 
])

pipeline4.fit(X_train, y_train)

y_preds3 = pipeline4.predict(X_train)
y_val_preds3 = pipeline4.predict(X_val)
y_test_preds3 = pipeline4.predict(X_test)

In [26]:
print_regressor_scores(y_preds3, y_train, set_name='Training')
print_regressor_scores(y_val_preds3, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds3, y_actuals=y_test, set_name='Testing')

RMSE Training: 239.69478899890268
MAE Training: 192.6699526841691
RMSE Validation: 239.48726029444512
MAE Validation: 192.60243057179738
RMSE Testing: 239.75430269125368
MAE Testing: 192.68703559240902


In [31]:
from sklearn.ensemble import RandomForestRegressor

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', RandomForestRegressor(
        n_estimators=20,
        max_depth=5,
        max_features='log2',
        random_state=42
    ))
])

pipeline_rf.fit(X_train, y_train)

y_preds5 = pipeline_rf.predict(X_train)
y_val_preds5 = pipeline_rf.predict(X_val)
y_test_preds5 = pipeline_rf.predict(X_test)

In [34]:
print_regressor_scores(y_preds5, y_train, set_name='Training')
print_regressor_scores(y_val_preds5, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds5, y_actuals=y_test, set_name='Testing')

RMSE Training: 153.66209223155462
MAE Training: 110.71380431801786
RMSE Validation: 152.62209640108296
MAE Validation: 110.6031855614612
RMSE Testing: 153.5224195293117
MAE Testing: 110.76359140367751


In [35]:
from sklearn.ensemble import RandomForestRegressor

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', RandomForestRegressor(
        n_estimators=20,
        max_depth=5,
        max_features='sqrt',
        random_state=42
    ))
])

pipeline_rf.fit(X_train, y_train)

y_preds5 = pipeline_rf.predict(X_train)
y_val_preds5 = pipeline_rf.predict(X_val)
y_test_preds5 = pipeline_rf.predict(X_test)

In [36]:
print_regressor_scores(y_preds5, y_train, set_name='Training')
print_regressor_scores(y_val_preds5, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds5, y_actuals=y_test, set_name='Testing')

RMSE Training: 148.66311192808527
MAE Training: 106.95499720718799
RMSE Validation: 147.63094347906735
MAE Validation: 106.86490898985168
RMSE Testing: 148.62350695127662
MAE Testing: 107.0206381875522


In [37]:
from sklearn.ensemble import RandomForestRegressor

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', RandomForestRegressor(
        n_estimators=50,
        max_depth=5,
        max_features='sqrt',
        random_state=42
    ))
])

pipeline_rf.fit(X_train, y_train)

y_preds5 = pipeline_rf.predict(X_train)
y_val_preds5 = pipeline_rf.predict(X_val)
y_test_preds5 = pipeline_rf.predict(X_test)

In [38]:
print_regressor_scores(y_preds5, y_train, set_name='Training')
print_regressor_scores(y_val_preds5, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds5, y_actuals=y_test, set_name='Testing')

RMSE Training: 147.3470493157371
MAE Training: 105.14035042981885
RMSE Validation: 146.3135251874094
MAE Validation: 105.06231659574148
RMSE Testing: 147.31635763754997
MAE Testing: 105.20337972359265


Choose linear regression, make performances better

In [44]:
from sklearn.linear_model import ElasticNet

elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.5)

pipeline_ela = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('elastic', ElasticNet(alpha=1.0, l1_ratio=0.5)) 
])

pipeline_ela.fit(X_train, y_train)

y_preds6 = pipeline_ela.predict(X_train)
y_val_preds6 = pipeline_ela.predict(X_val)
y_test_preds6 = pipeline_ela.predict(X_test)

In [45]:
print_regressor_scores(y_preds6, y_train, set_name='Training')
print_regressor_scores(y_val_preds6, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds6, y_actuals=y_test, set_name='Testing')

RMSE Training: 161.54868516178252
MAE Training: 108.85429637746299
RMSE Validation: 160.2458962104936
MAE Validation: 108.76579881143603
RMSE Testing: 161.8102993976421
MAE Testing: 108.94793961291643


In [17]:
from sklearn.preprocessing import PolynomialFeatures

pipeline_poly = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', PolynomialFeatures(degree=2)),  # Add polynomial features
    ('regressor', LinearRegression())
])


pipeline_poly.fit(X_train, y_train)

y_preds6 = pipeline_poly.predict(X_train)
y_val_preds6 = pipeline_poly.predict(X_val)
y_test_preds6 = pipeline_poly.predict(X_test)

In [18]:
print_regressor_scores(y_preds6, y_train, set_name='Training')
print_regressor_scores(y_val_preds6, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds6, y_actuals=y_test, set_name='Testing')

RMSE Training: 126.53116727149795
MAE Training: 88.94387358432162
RMSE Validation: 125.68721603026289
MAE Validation: 88.86830659525043
RMSE Testing: 126.13707848658244
MAE Testing: 88.96173089218289


In [18]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

pipeline_ridge = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', PolynomialFeatures(degree=2)),  # Add polynomial features
    ('regressor', Ridge(alpha=0.01))
])


pipeline_ridge.fit(X_train, y_train)

y_preds6 = pipeline_ridge.predict(X_train)
y_val_preds6 = pipeline_ridge.predict(X_val)
y_test_preds6 = pipeline_ridge.predict(X_test)

In [19]:
print_regressor_scores(y_preds6, y_train, set_name='Training')
print_regressor_scores(y_val_preds6, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds6, y_actuals=y_test, set_name='Testing')

RMSE Training: 126.53138591013736
MAE Training: 88.94318428414516
RMSE Validation: 125.68718968060071
MAE Validation: 88.86738462926364
RMSE Testing: 126.13731044947077
MAE Testing: 88.96105398001401
